# 3.6 Softmax regression from scratch

In [2]:
from IPython import display
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

In [3]:
batch_size=256
train_iter, test_iter=d2l.load_data_fashion_mnist(batch_size)

In [4]:
num_inputs=784
num_outputs=10

w=np.random.normal(0,0.01,(num_inputs,num_outputs))
b=np.zeros(num_outputs)
w.attach_grad()
b.attach_grad()

In [5]:
X=np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])
X.shape,X.sum(0,keepdims=True),X.sum(1,keepdims=True)

((2, 3),
 array([[5., 7., 9.]]),
 array([[ 6.],
        [15.]]))

In [6]:
def softmax(X):
    X_exp=np.exp(X)
    partition = X_exp.sum(1,keepdims=True)
    return X_exp/partition

In [7]:
X = np.random.normal(0,1,(2,5))
X_prob = softmax(X)
X,X_prob, X_prob.sum(1)

(array([[ 0.5030464 , -0.7089397 , -0.72034746,  0.79505837,  0.9336002 ],
        [ 1.0187219 , -1.97108   , -0.15738758, -0.907509  , -1.5379014 ]]),
 array([[0.22376052, 0.06659239, 0.06583703, 0.29964197, 0.3441681 ],
        [0.63209665, 0.03179282, 0.194987  , 0.09209415, 0.04902935]]),
 array([1.        , 0.99999994]))

In [8]:
def net(X):
    return softmax(np.dot(X.reshape((-1,W.shape[0])),W)+b)

# Loss function

In [27]:
y=np.array([0,2])
y_hat=np.array([[0.1,0.3,0.6],[0.4,0.2,0.5]])
y_hat[[0,1],y]

array([0.1, 0.5])

In [29]:
y_hat[[0,0],[1,0]]

array([0.3, 0.1])

In [30]:
def cross_entropy(y_hat,y):
    return -np.log(y_hat[range(len(y_hat)),y])

cross_entropy(y_hat,y)

array([2.3025851, 0.6931472])